# Clustering  con K-means - Catado de cafe
Se tiene una base de datos con los resultados de diferentes catados de multiples muestras de cafe. 

Se desea realizar un agrupamiento de éstas muestras según sus métricas estadísticas. Entre ellas se encuentra la calificación promedio del catador certificado y niveles de sabor: Vainilla, floral, cereral, cocoa, alcohol, fermentado, tostado, oscuro, amargo, entre otros.

En este caso se hace uso del algorítmo KMeans, que se explica más adelante.

In [ ]:
# Imports

import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import pandas as pd

%matplotlib inline

## Importando Dataset y visualizando sus Características

In [ ]:
cafes = pd.read_csv('../datasets/catacafe.csv',engine='python')
cafes.info()

## Mostrar las primeras filas para una previsualización del orden de los datos

In [ ]:
cafes.head()

## Se puede ver que la columna "Cafe" es el índice numérico de la muestra, pero que ya se ha enumerado con el método de lectura, por ello, se procede a eliminarla.

In [ ]:
cafes = cafes.drop(['Cafe'],axis=1)
cafes.head()


## Se obtienen las variables estadísticas de los datos por columna.

In [ ]:
cafes.describe()

## Se normalizan los datos en un rango adecuado y se vuelven a obtener sus métricas

In [ ]:
cafes_normalizado = (cafes - cafes.min())/(cafes.max()-cafes.min())
cafes_normalizado.describe()

Se puede observar que ahora los datos tienen valores entre cero y uno, max y min.

## Ahora con este preprocesamiento se tienen datos ordenados, numéricos y normalizados, listos para un agrupamiento óptimo.
El método de KMeans presenta gran efectividad y velocidad para datos no tan amplios, sin embargo, su principal debilidad es la selección de parámetros de entrada, entiéndase número de clusters a realizar. Como este valor no se conoce, se debe usar algún método para óptimizar su implementación, en este caso se utilizará el método del codo de Jambú para encontrar un número de clusters óptimo:

## Obtencion del gráfico del Codo de Jambú

Se desean que los clusteres sean lo más separados entre sí y que sus elementos sean lo más cercanos entre sí. para ello se utiliza la medida WCSS: suma de los cuadrados de las distancias de cada punto de datos, en todos los grupos a sus respectivos centroides, es decir, es una medida de similitud.
La idea es minimizar esta suma. Para ello se obtiene la inercia de cada clustering realizado con KMeans para un cierto número de grupos, desde 1 hasta uno deseado, estos valores obtenidos en cada iteración se almacena en WCSS, donde luego se imprimen en una gráfica para su análisis.

In [ ]:
num_clusters = 10
wcss = []
for i in range(1,num_clusters+1):
    kmeans_model = KMeans(n_clusters=i,max_iter=300)
    kmeans_model.fit(cafes_normalizado)
    wcss.append(kmeans_model.inertia_)

## Ahora se grafican los resultados:

plt.plot(range(1,num_clusters+1),wcss)
plt.title("Codo de Jambú")
plt.xlabel("Número de Clusters")
plt.ylabel("WCSS")
plt.show()

Se observa que le número de clusteres optimo es 3, para este metodo y este dataset. Ahora se procede a utilizar el metodo Kmeans con este parametro. Igual que anteriormente, se crea el modelo de clustering y luego se aplica con .fit

In [ ]:
agrupamiento =  KMeans(n_clusters=3, max_iter=300) 
agrupamiento.fit(cafes_normalizado)

Este metodo crea un atributo label_ dentro del modelo clustering generado. Se agrega esta calificacion al archivo original del Dataset:

In [ ]:
cafes['KMeans_clusters'] = agrupamiento.labels_
cafes.head()

## Visualización de los clusters Generados
Los datos tienen múltiples variables que los caracterizan, en este caso se desea visualizar un gráfico lo mayor resumido posible, y en la naturaleza humana se alcanzan a visualizar hasta tres dimensiones.

Para efectos didácticos, se mostraran en dos dimensiones ¿Cuales? se seleccionan las variables que mejor caractericen a todos los datos, para ello se hace uso del Análisis de Componentes Principales (PCA) para reducir el número de variables a analizar, en este caso a visualizar.
Se hace uso del paquete descomposition de sklearn y se crea un dataframe a partir de estos componentes para graficarlo.

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)                         # Dos componentes principales
pca_cafes = pca.fit_transform(cafes_normalizado)
pca_cafes_df = pd.DataFrame(data= pca_cafes, columns=['Componente_1', 'Componente_2'])
pca_names_cafes = pd.concat([pca_cafes_df, cafes[['KMeans_clusters']]], axis=1)

# veamos el resultado de los datos procesados:
pca_names_cafes  

## Graficar el dataframe procesado
Ahora se configura la figura plot a mostrar con estos datos obtenidos

In [ ]:
# Configurando la figura plot
fig = plt.figure(figsize= (7,7))                                                    # área del gráfico
grafico = fig.add_subplot(1,1,1)                                                    # Se delimita el área subplot a una sola figura
grafico.set_xlabel('Componente 1',fontsize = 12 )                                   # Etiqueta de eje X y tamaño de letra
grafico.set_ylabel('Componente 2',fontsize = 12 )                                   # Etiqueta de eje Y y tamaño de letra
grafico.set_title('Componentes Principales - Clustering Kmeans',fontsize = 20 )     # Setear el título de la figura
Colores = np.array(["blue", "orange", "green"])                                     # Vector de nombres de colores a llamar. Deben haber tantos colores como clusters(etiquetas) generados 
grafico.scatter(x=pca_names_cafes.Componente_1, y=pca_names_cafes.Componente_2, c=Colores[pca_names_cafes.KMeans_clusters], s=40)   #llamado al método de figura de puntos de dispersión.
plt.show()                                                                          # Graficar

## Guardar los datos generados
Se procede a guardar el dataframe en formato csv:

In [ ]:
# Se crea un archivo csv en la carpeta Results
cafes.to_csv('../Results/cafe-kmeans.csv')

# Ejercicios/ Experimentos propuestos

## 1. Elija un valor aleatorio para el número de clusters a implementar, suponiendo que no conoce el resultado del método Codo de Jambú. ¿Cómo cambia el resultado? ¿Qu+e se nota?
## 2. ¿Qué sucede al aumentar o disminuir el número de clusters a implementar en la llamada a KMeans? ¿Por qué?
## 3. ¿Qué sucede con los clusters al aumentar o disminuir significativamente el número de iteraciones máximo (seteado en 300) al usar el metodo del Codo de Jambú y en la llamada a KMeans? ¿Por qué?
## 4. Explique las ventajas y desventajas que tiene el algoritmo KMeans. Puede investigar diferentes fuentes.
## 5. Aumente el número de PCA a 3 componentes y grafíquelo en 3 Dimensiones. ¿Qué es lo que cambió y qué se está añadiendo?

## Experimentos a realizar
Aumentar las variables PCA a 3, configurando el grafico para mostrarlo.

In [ ]:
pca = PCA(n_components=3)
pca_cafes = pca.fit_transform(cafes_normalizado)
pca_cafes_df = pd.DataFrame(data= pca_cafes, columns=['Componente_1', 'Componente_2','Componente_3'])
pca_names_cafes = pd.concat([pca_cafes_df, cafes[['KMeans_clusters']]], axis=1)

pca_names_cafes   # veamos el resultado de los datos procesados:

In [ ]:
# Graficamos este nuevo dataframe en 3D
#from mpl_toolkits.mplot3d import Axes3D  #otra forma de plotearlo, permite interactividad
#grafico = Axes3D(fig)

fig = plt.figure(figsize= (12,12))
grafico = fig.add_subplot(111, projection='3d')

x = pca_names_cafes.Componente_1
y = pca_names_cafes.Componente_2
z = pca_names_cafes.Componente_3

grafico.set_xlabel('Componente 1',fontsize = 12 )
grafico.set_ylabel('Componente 2',fontsize = 12 )
grafico.set_zlabel('Componente 3',fontsize = 12 )
grafico.set_title('3 Componentes principales - Clustering Kmeans',fontsize = 20 )
Colores = np.array(["blue", "orange", "green"])

# Agregamos los puntos en el plano 3D
grafico.scatter(x, y, z, c=Colores[pca_names_cafes.KMeans_clusters], marker='o')
plt.show()

# Como se puede observar, se está agregando la tercer componente más representativa del conjunto de datos, obtenida por medio de PCA, alcanzando la maxima visualizacion posible.
# Es posible mover el gráfico 3D en algunos visualizadores y observar mejor la relacion entre los diferentes clusters.